In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import Lars
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from IPython.display import Image

import category_encoders as ce
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import ShuffleSplit
from scipy.stats.mstats import winsorize
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [6]:
# Блок(и) обучения и поверки модели

In [7]:
df = pd.read_csv(path_train)
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.01,Ideal,I,SI2,62.1,56.0,6.39,6.41,3.98,4078
1,1.08,Ideal,H,VS2,62.7,55.0,6.53,6.59,4.09,5851
2,2.02,Premium,I,VS2,59.8,60.0,8.20,8.28,4.92,16383
3,0.32,Very Good,G,SI1,63.0,56.0,4.33,4.36,2.74,477
4,0.74,Good,E,SI2,63.4,64.0,5.76,5.71,3.63,2401
...,...,...,...,...,...,...,...,...,...,...
145174,0.50,Very Good,E,SI1,60.2,61.0,5.11,5.15,3.09,1410
145175,1.91,Very Good,F,SI1,62.3,62.0,7.85,7.79,4.87,15064
145176,1.22,Premium,G,VS2,62.8,58.0,6.82,6.74,4.26,7209
145177,0.31,Very Good,G,VVS1,61.1,56.0,4.37,4.40,2.67,816


In [8]:
# ЭТОТ БЛОК ПОСВЯЩЕН НАХОЖДЕНИЮ НЕПРАВИЛЬНЫХ ЗНАЧЕНИЙ В X,Y,Z В TRAIN_DATA. БЛОК ПОМЕЩАЕТСЯ В НАЧАЛО РАБОТЫ
# отбор всех сторок с нулевым. значением в любом месте колонки
display(df[df.eq(0).any(axis=1)])
print('Еще увидим значение 1.05')
# Видели еще значение больше 0, но выделяющиеся проверим
display(df[df['z']<2])
print("Еще увидим значения в z> 6.5")
display(df[df['z']>6.5]) # тоже считаются выбросами значения больше 6.5, в x,y - это норма, но в z - нет, поэтому мы их
# заменим также через расчет z=(depth*mean(x,y))/100

# заменяем там где можно подсчитать математически
df.loc[28099, 'z'] = 5.1
df.loc[81538, 'z'] = 5.1
df.loc[110571, 'z']= 5.01
df.loc[127686, 'z'] = 5.1
df.loc[41748,'z'] = 4.04

df.loc[80198,'z'] = 5.1
df.loc[88644,'z'] = 4.87
df.loc[119299,'z'] = 5.15
df.loc[124083,'z'] = 3.13 # было 31.30 , опечатались

print("Оставшиеся строки, к которым мы будем применять KNNImputer на след шагах:")
display(df[df.eq(0).any(axis=1)])


,carat,cut,color,clarity,depth,table,x,y,z,price
28099,2.17,Premium,H,SI2,60.3,57.0,8.42,8.36,0.0,15923
44199,0.71,Very Good,F,SI2,62.0,60.0,0.00,6.71,0.0,2130
81538,2.20,Premium,I,SI2,60.1,60.0,8.45,8.41,0.0,11221
95099,0.71,Good,F,SI1,64.1,60.0,0.00,0.00,0.0,2130
110571,2.18,Premium,H,SI2,59.4,60.0,8.46,8.41,0.0,15842
127686,2.18,Premium,H,SI2,60.8,59.0,8.42,8.38,0.0,13938


Еще увидим значение 1.05


,carat,cut,color,clarity,depth,table,x,y,z,price
28099,2.17,Premium,H,SI2,60.3,57.0,8.42,8.36,0.00,15923
41748,1.07,Ideal,H,SI1,61.3,57.0,6.57,6.62,1.05,5125
44199,0.71,Very Good,F,SI2,62.0,60.0,0.00,6.71,0.00,2130
81538,2.20,Premium,I,SI2,60.1,60.0,8.45,8.41,0.00,11221
95099,0.71,Good,F,SI1,64.1,60.0,0.00,0.00,0.00,2130
110571,2.18,Premium,H,SI2,59.4,60.0,8.46,8.41,0.00,15842
127686,2.18,Premium,H,SI2,60.8,59.0,8.42,8.38,0.00,13938


Еще увидим значения в z> 6.5


,carat,cut,color,clarity,depth,table,x,y,z,price
80198,2.10,Premium,H,I1,61.1,57.0,8.42,8.28,8.35,12512
88644,2.01,Premium,I,VS1,59.1,60.0,8.28,8.21,8.18,17640
119299,2.22,Ideal,G,VS2,61.8,57.0,8.36,8.32,8.40,18736
124083,0.51,Premium,G,VS2,61.8,58.0,5.20,5.13,31.30,1447


Оставшиеся строки, к которым мы будем применять KNNImputer на след шагах:


,carat,cut,color,clarity,depth,table,x,y,z,price
44199,0.71,Very Good,F,SI2,62.0,60.0,0.0,6.71,0.0,2130
95099,0.71,Good,F,SI1,64.1,60.0,0.0,0.00,0.0,2130


In [9]:
train_data = df.copy()

train_data['price'] = train_data['price'].apply(np.sqrt)
train_data

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.01,Ideal,I,SI2,62.1,56.0,6.39,6.41,3.98,63.859220
1,1.08,Ideal,H,VS2,62.7,55.0,6.53,6.59,4.09,76.491830
2,2.02,Premium,I,VS2,59.8,60.0,8.20,8.28,4.92,127.996094
3,0.32,Very Good,G,SI1,63.0,56.0,4.33,4.36,2.74,21.840330
4,0.74,Good,E,SI2,63.4,64.0,5.76,5.71,3.63,49.000000
...,...,...,...,...,...,...,...,...,...,...
145174,0.50,Very Good,E,SI1,60.2,61.0,5.11,5.15,3.09,37.549967
145175,1.91,Very Good,F,SI1,62.3,62.0,7.85,7.79,4.87,122.735488
145176,1.22,Premium,G,VS2,62.8,58.0,6.82,6.74,4.26,84.905830
145177,0.31,Very Good,G,VVS1,61.1,56.0,4.37,4.40,2.67,28.565714


In [10]:
train_data['carat'] = np.log(train_data['carat'])

In [11]:
def round_table_values(x):
    if x % 1 == 0:
        return x
    elif x % 1 <= 0.3:
        return np.floor(x)
    elif x % 1 > 0.3 and x % 1 <= 0.7:
        return np.floor(x) + 0.5
    else:
        return np.ceil(x)

# применяем функцию к колонке 'table' в dataframe 'train_data'
train_data['table'] = train_data['table'].apply(round_table_values)

In [12]:
simpleimputer=  SimpleImputer(missing_values = 0, strategy = 'mean')

train_data[['x', 'y', 'z']] = simpleimputer.fit_transform(train_data[['x', 'y', 'z']])

train_data['y_z'] = train_data['y']/train_data['z']

train_data['y_z'] = simpleimputer.fit_transform(train_data[['y_z']])



In [13]:
X = train_data.drop(['price'],axis=1)
y = train_data['price']

In [14]:
axis_features = ['x','y','z']
num_features = ['carat','depth','table','y_z']

axis_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=0, strategy='mean',add_indicator=False)),
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', MinMaxScaler())])

clarity_map = [{
    'col':'clarity',##### Обратить внимание
    'mapping':{'FL':10, 'IF':9, 'VVS1':8, 'VVS2':7, 'VS1':6, 'VS2':5, 
               'SI1':4, 'SI2':3, 'I1':2, 'I2':1, 'I3':0}
    }]


clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])
      
    
color_map = [{
    'col':'color',
    'mapping':{'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}
    }]

color_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=color_map)),
    ('scaler', MinMaxScaler())
    ])

cut_map = [
    {'col':'cut',
    'mapping':{"Fair":0, "Good":1, "Very Good":2, "Premium":3,"Ideal":4}
    }]

cut_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=cut_map)),
    ('scaler', MinMaxScaler())
    ])
    

CT = ColumnTransformer([
        ("pol_std", axis_transformer, axis_features),
        ("num", MinMaxScaler(), num_features),
        ("ordinal_map_color", color_transformer, ['color']) ,
        ("ordinal_map_cut", cut_transformer, ['cut']) ,
        ("ordinal_map", clarity_transformer, ['clarity'])], 
#         ('carat_bin', MinMaxScaler(), ['carat_bin','figsize'])],
        remainder = 'passthrough')

display(CT)

res_ct = CT.fit_transform(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pol_std',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(missing_values=0)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['x', 'y', 'z']),
                                ('num', MinMaxScaler(),
                                 ['carat', 'depth', 'table', 'y_z']),
                                ('ordinal_map_color',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col':...
                                                  OrdinalEncoder(mapping=[{'col': 'cut',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': Fair         0
Good         1
Very Good    2
Premium      3
Ideal        4
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['cut']),
                                ('ordinal_map',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': FL      10
IF       9
VVS1     8
VVS2     7
VS1      6
VS2      5
SI1      4
SI2      3
I1       2
I2       1
I3       0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['clarity'])])

Работаем с тестовым набором

In [15]:
df_test = pd.read_csv(path_test)
df_test

,carat,cut,color,clarity,depth,table,x,y,z
0,0.50,Ideal,D,SI1,62.1,57.0,5.05,5.08,3.14
1,2.00,Very Good,G,SI2,59.5,57.0,8.08,8.15,4.89
2,0.27,Premium,E,VVS1,60.5,59.0,4.19,4.16,2.52
3,0.32,Premium,I,VVS1,61.2,59.0,4.43,4.44,2.71
4,1.19,Ideal,H,SI1,62.5,56.0,6.77,6.81,4.23
...,...,...,...,...,...,...,...,...,...
48389,0.43,Very Good,D,VS2,63.2,57.0,4.84,4.81,3.04
48390,2.02,Premium,J,SI2,62.4,60.0,8.03,8.10,5.03
48391,0.39,Very Good,G,SI1,63.5,56.0,4.58,4.64,2.95
48392,0.53,Ideal,G,VVS1,61.2,56.0,5.25,5.22,3.20


In [16]:
# БЛОК ПОСВЯЩЕН TEST_DATA НАХОЖДЕНИЮ НЕПРАВИЛЬНЫХ ЗНАЧЕНИЙ И ИХ ЗАМЕНА
# отбор всех сторок с нулевым. значением в любом месте колонки
display(df_test[df_test.eq(0).any(axis=1)])

# заменяем там где можно подсчитать математически
df_test.loc[7724, 'z'] = 5.15
df_test.loc[16136, 'z'] = 3.9
df_test.loc[44526, 'z']= 5.1

print("Оставшиеся строки, к которым мы будем применять KNNImputer:")
display(df_test[df_test.eq(0).any(axis=1)])

# 7724 - z= 5.14692 ~~ 5.15
# 16136 - z= 3.8907 ~ 3.9
# 44526 - z= 5.107 ~ 5.1

,carat,cut,color,clarity,depth,table,x,y,z
4602,0.71,Good,D,VS2,64.1,60.0,0.00,0.00,0.0
7724,2.18,Premium,I,VS2,61.2,62.0,8.45,8.37,0.0
16136,1.02,Premium,H,SI2,59.4,61.0,6.57,6.53,0.0
44526,2.20,Premium,F,SI2,60.3,58.0,8.49,8.45,0.0


Оставшиеся строки, к которым мы будем применять KNNImputer:


,carat,cut,color,clarity,depth,table,x,y,z
4602,0.71,Good,D,VS2,64.1,60.0,0.0,0.0,0.0


In [17]:

df_test['carat'] = np.log(df_test['carat'])
df_test['table'] = df_test['table'].apply(round_table_values)


In [18]:
# simpleimputer=  SimpleImputer(missing_values = 0, strategy = 'mean')

df_test[['x', 'y', 'z']] = simpleimputer.fit_transform(df_test[['x', 'y', 'z']])

df_test['y_z'] = df_test['y']/df_test['z']

df_test['y_z'] = simpleimputer.fit_transform(df_test[['y_z']])



In [19]:
df_test

,carat,cut,color,clarity,depth,table,x,y,z,y_z
0,-0.693147,Ideal,D,SI1,62.1,57.0,5.05,5.08,3.14,1.617834
1,0.693147,Very Good,G,SI2,59.5,57.0,8.08,8.15,4.89,1.666667
2,-1.309333,Premium,E,VVS1,60.5,59.0,4.19,4.16,2.52,1.650794
3,-1.139434,Premium,I,VVS1,61.2,59.0,4.43,4.44,2.71,1.638376
4,0.173953,Ideal,H,SI1,62.5,56.0,6.77,6.81,4.23,1.609929
...,...,...,...,...,...,...,...,...,...,...
48389,-0.843970,Very Good,D,VS2,63.2,57.0,4.84,4.81,3.04,1.582237
48390,0.703098,Premium,J,SI2,62.4,60.0,8.03,8.10,5.03,1.610338
48391,-0.941609,Very Good,G,SI1,63.5,56.0,4.58,4.64,2.95,1.572881
48392,-0.634878,Ideal,G,VVS1,61.2,56.0,5.25,5.22,3.20,1.631250


In [20]:
# Блок предсказания с использованием тестового набора

In [21]:
pipeline_best = Pipeline([('preprocessing', CT),
                ('regression', RandomForestRegressor(max_features = 13, min_samples_leaf = 5, n_estimators = 150
                                                    ,criterion = 'friedman_mse', min_samples_split = 3,
                                                    n_jobs = -1))])
pipeline_best.fit(X, y)

y_train_pred = np.square(pipeline_best.predict(X))

print("Train:",mean_squared_error(np.square(y), y_train_pred))

y_predict = np.square(pipeline_best.predict(df_test))

Train: 179243.3528862547


In [22]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict

array([ 1423.95918263, 14542.0795486 ,   699.42684464, ...,
         643.8181907 ,  2010.19813317,   839.31133113])